# Model development of salary classification of Census data


In [83]:
# model.py 
from sklearn.metrics import fbeta_score, precision_score, recall_score


# Load data 
def load_data(data_frame):
    """ Load our cleaned csv file
    """
    df = pd.read_csv(data_frame)
    
    return df


# Optional: implement hyperparameter tuning.
def train_model(X_train, y_train, ):
    """
    Trains a machine learning model and returns it.

    Inputs
    ------
    X_train : np.array
        Training data.
    y_train : np.array
        Labels.
    Returns
    -------
    model
        Trained machine learning model.
    """
    # Check 3 classifiers results and take the best results 
    log_clf = LogisticRegression()
    rnd_clf = RandomForestClassifier()
    svm_clf = SVC(

    vorting_clf = VotingClassifier(
        estimatros = [('lr', log_clf),('rf', rnd_clf),('svc', svm_clf)],
        voting='hard' )
    voting_clf.fit(X_train, y_train)

    


def compute_model_metrics(y, preds):
    """
    Validates the trained machine learning model using precision, recall, and F1.

    Inputs
    ------
    y : np.array
        Known labels, binarized.
    preds : np.array
        Predicted labels, binarized.
    Returns
    -------
    precision : float
    recall : float
    fbeta : float
    """
    fbeta = fbeta_score(y, preds, beta=1, zero_division=1)
    precision = precision_score(y, preds, zero_division=1)
    recall = recall_score(y, preds, zero_division=1)
    return precision, recall, fbeta


def inference(model, X):
    """ Run model inferences and return the predictions.

    Inputs
    ------
    model : ???
        Trained machine learning model.
    X : np.array
        Data used for prediction.
    Returns
    -------
    preds : np.array
        Predictions from the model.
    """
    pass



In [ ]:
def one_hot_encode_categorical(df, )

In [140]:
# data

def process_data(
    X, categorical_features=[], label=None, training=True, encoder=None, lb=None
):
    """ Process the data used in the machine learning pipeline.

    Processes the data using one hot encoding for the categorical features and a
    label binarizer for the labels. This can be used in either training or
    inference/validation.

    Note: depending on the type of model used, you may want to add in functionality that
    scales the continuous data.

    Inputs
    ------
    X : pd.DataFrame
        Dataframe containing the features and label. Columns in `categorical_features`
    categorical_features: list[str]
        List containing the names of the categorical features (default=[])
    label : str
        Name of the label column in `X`. If None, then an empty array will be returned
        for y (default=None)
    training : bool
        Indicator if training mode or inference/validation mode.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained sklearn OneHotEncoder, only used if training=False.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained sklearn LabelBinarizer, only used if training=False.

    Returns
    -------
    X : np.array
        Processed data.
    y : np.array
        Processed labels if labeled=True, otherwise empty np.array.
    encoder : sklearn.preprocessing._encoders.OneHotEncoder
        Trained OneHotEncoder if training is True, otherwise returns the encoder passed
        in.
    lb : sklearn.preprocessing._label.LabelBinarizer
        Trained LabelBinarizer if training is True, otherwise returns the binarizer
        passed in.
    """

    if label is not None:
        y = X[label]
        X = X.drop([label], axis=1)
    else:
        y = np.array([])

    X_categorical = X[categorical_features].values
    X_continuous = X.drop(*[categorical_features], axis=1)

    if training is True:
        encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
        lb = LabelBinarizer()
        X_categorical = encoder.fit_transform(X_categorical)
        y = lb.fit_transform(y.values).ravel()
    else:
        X_categorical = encoder.transform(X_categorical)
        try:
            y = lb.transform(y.values).ravel()
        # Catch the case where y is None because we're doing inference.
        except AttributeError:
            pass

    X = np.concatenate([X_continuous, X_categorical], axis=1)
    return X, y, encoder, lb


## Load libraries

In [144]:
# Loading packages 
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC



## Data Preprocessing
* Load cleaned Census data
* OneHot encode categorical features
* label binarize the label

In [142]:
model_dir = "../model/"
df = pd.read_csv("../data/census_cleaned.csv", index_col=None)
df.head()


# Categorical features 
cat_features_list = ['workclass',
                     'education',
                     'marital-status',
                     'occupation',
                     'relationship',
                     'race',
                     'sex',
                     'native-country']
# numerical features
num_feature_list = ['age',
                    'fnlgt',
                    'education-num',
                    'capital-gain',
                    'capital-loss', 
                    'hours-per-week']

print("Loaded Census data set")
census_df = load_data("../data/census_cleaned.csv")

X = census_df

Loaded Census data set


## Encode categorical features

In [143]:
X, y, encoder, lb = process_data(X, cat_features_list, label='salary', 
                                 training=True, encoder=OneHotEncoder, lb=LabelBinarizer)

In [59]:
# preprocessing 
categorical_features = X.select_dtypes(exclude=[np.number])
cat_features_list = list(categorical_features.columns.values)
print(cat_features_list)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


In [76]:
# preprocessing 
num_features = X.select_dtypes(include=[np.number])
num_features_list = list(num_features.columns.values)
print(num_features_list)

['age', 'fnlgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


In [39]:
X_categorical = X[cat_features_list]
X_categorical

,workclass,education,marital-status,occupation,relationship,race,sex,native-country
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba
...,...,...,...,...,...,...,...,...
30157,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States
30158,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
30159,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States
30160,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States


In [40]:
encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
lb = LabelBinarizer()
X_categorical = encoder.fit_transform(X_categorical)

In [46]:
X_categorical.shape

(30162, 98)

In [147]:
pwd

'/Users/obara/model-deployment-heroku-fastapi/starter/notebooks'

In [149]:
df_cleaned_test = pd.read_csv("../data/census_cleaned_test.csv")
df_cleaned_test.head()

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,41,Local-gov,174575,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
1,22,Private,107882,HS-grad,9,Never-married,Prof-specialty,Own-child,White,Female,0,0,20,United-States,<=50K
2,79,Local-gov,84616,11th,7,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,7,United-States,<=50K
3,44,Private,191256,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K
4,33,Private,319854,Bachelors,13,Separated,Prof-specialty,Not-in-family,White,Male,4650,0,35,United-States,<=50K


In [150]:
df_cleaned_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6033 entries, 0 to 6032
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             6033 non-null   int64 
 1   workclass       6033 non-null   object
 2   fnlgt           6033 non-null   int64 
 3   education       6033 non-null   object
 4   education-num   6033 non-null   int64 
 5   marital-status  6033 non-null   object
 6   occupation      6033 non-null   object
 7   relationship    6033 non-null   object
 8   race            6033 non-null   object
 9   sex             6033 non-null   object
 10  capital-gain    6033 non-null   int64 
 11  capital-loss    6033 non-null   int64 
 12  hours-per-week  6033 non-null   int64 
 13  native-country  6033 non-null   object
 14  salary          6033 non-null   object
dtypes: int64(6), object(9)
memory usage: 707.1+ KB
